In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm   #for文進捗の可視化
import time

N=30000  #検索id数、30000が妥当 ◇全データ取得の場合はこちら◇
k=0
id_list = [21470,15164,24348,9454,25768,12054,5756,22306,9714,14155,15093,21462,10821,16474,10081,12874,12050,26049,12061,7421,21603,21161,18261,13174,24758,20575,20272,17470,17469,18984] #◇id検索の場合はこちら◇

df_REP=pd.DataFrame()

# for ParkingId in tqdm(range(N)): #◇全データ取得の場合はこちら◇
for ParkingId in tqdm(id_list): #◇id検索の場合はこちら◇

    request_url = "https://www.repark.jp/parking_user/time/result/detail/?park=REP" + str(ParkingId).zfill(7)

    res = requests.get(request_url)
    res.encoding = 'utf-8'

    # 初期化
    Name = ""
    Address = ""
    Charge = ""
    Status = ""
    ParkingSpaces = ""
    Restriction = ""
    Hours = ""
    Latitude = ""
    Longitude = ""

    if res.status_code==200:
        soup = BeautifulSoup(res.text, 'html.parser')

        ##############駐車場ID###############
        # print(ParkingId)
        ParkingId="REP"+str(ParkingId)

        ##############駐車場名###############
        Name = soup.select('h1.page-title')[0].text
        # print(Name)

        ##############住所###############
        Address=soup.select('tbody td')[1].text
        # print(Address)

        ##############料金###############
        Rowcount = str(soup.select('div.lv2')).count('<div class="section lv3">')
        Rowcount2 = str(soup.select('div.lv3')).count('</p>')
        Charge=""
        i=0
        j=0
        for i in range(Rowcount):
            Charge+=soup.select('div.lv3')[i].select('h3')[0].text + "\n"

            for j in range(min(Rowcount2, len(soup.select('div.lv3')[i].select('p')))):
                Charge+=soup.select('div.lv3')[i].select('p')[j].text + "\n"

        Charge+=soup.select('div.item p')[0].text

        ##############空満情報###############
        Status=soup.select('span.icon img')[0]["alt"]
        # print(Status)

        ##############駐車台数###############
        ParkingSpaces=soup.select('tbody td')[3].text
        # print(ParkingSpaces)

        ##############駐車制限###############
        Restriction=soup.select('tbody td')[4].text
        # print(Restriction)

        ##############営業時間###############
        Hours=soup.select('tbody td')[2].text
        # print(Hours)

        ##############緯度###############
        Latitude = soup.select('input#latitude')[0]["value"]
        # print(Latitude)

        ##############経度###############
        Longitude = soup.select('input#longitude')[0]["value"]
        # print(Longitude)

        infolist={"ParkingId":ParkingId,"Name":Name,"Address":Address,"Charge":Charge,"Status":Status,"ParkingSpaces":ParkingSpaces,"Restriction":Restriction,"Hours":Hours,"Latitude":Latitude,"Longitude":Longitude}
        df=pd.DataFrame(infolist,index=[k])
        df_REP=pd.concat([df_REP,df])
        k+=1

    time.sleep(1)

100%|██████████| 30/30 [00:50<00:00,  1.68s/it]
